In [1]:
pip install bitsandbytes

In [2]:
pip install accelerate

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id =  "codellama/CodeLlama-13b-Instruct-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
eos_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
question = """write a javascript function to add 2 numbers"""

def evaluate(question):
  eval_prompt = """### Task
  # Generate a SQL query to answer the following question. Give answer according to MySql 5.7.21:
  # `{question}

  # ### Database Schema
  # This query will run on a database whose schema is represented in this string:
  # if all columns are required give the (*) notation.


  # CREATE TABLE IF NOT EXISTS orders (
  # <define your="" table="" schema="">;

  # ### SQL
  # Given the database schema, here is the SQL query that answers `{question}`:
  # ```sql
  # """.format(question=question)
  print(question)
  model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")


  import sqlparse
  outputs = tokenizer.batch_decode(model.generate(**model_input,eos_token_id=eos_token_id,pad_token_id=eos_token_id,max_new_tokens=100,do_sample=False,
      num_beams=1), skip_special_tokens=True)
  sql = sqlparse.format(outputs[0].split("```sql")[-1], reindent=True)

  return sql

sql = evaluate(question)

write a javascript function to add 2 numbers


In [9]:
import re
def refine_sql(sql):
  ans=''
  for i in range(0, len(sql)):
    if sql[i]!='#' and sql[i]!='`':
      ans+=sql[i]
    if sql[i] =='`':
      break
  sql=ans
  sql=re.sub(r'\s+$', '', sql)
  if_sql=True
  if sql=="":
    if_sql=False
  return sql, if_sql

sql, if_sql = refine_sql(sql)
print(if_sql)

False


In [10]:
# Used to securely store your API key
from google.colab import userdata

In [11]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [13]:
model1 = genai.GenerativeModel('gemini-pro')

In [14]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [15]:
def sql_to_gemini(sql, question, if_sql):
  prompt = f"""Write a javascript function that corresponds to the sql query : {sql}"""
  if if_sql==False:
    prompt = question
  response = model1.generate_content(prompt, stream=True)
  return response

response = sql_to_gemini(sql, question, if_sql)

In [16]:
def print_answer(response):
  for chunk in response:
    print(chunk.text)
print_answer(response)

```javascript
function add(num1, num2) {
  //
 Check if the arguments are valid numbers
  if (typeof num1 !== "number" || typeof num2 !== "number") {
    throw new Error
("Invalid arguments: both arguments must be numbers.");
  }

  // Add the two numbers and return the result
  return num1 + num2;
}
```


In [17]:
question = """
             Table: Employee
            Column Name: id, name, salary, departmentId
            Type: int, varchar, int, int

            id is the primary key (column with unique values) for this table.
            departmentId is a foreign key (reference column) of the ID from the Department table.
            Each row of this table indicates the ID, name, and salary of an employee. It also contains the ID of their department.


            Table: Department
            Column Name: id, name
            Type: int, varchar

            id is the primary key (column with unique values) for this table.
            Each row of this table indicates the ID of a department and its name.


            A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.

            Write a javascript function to find the employees who are high earners in each of the departments.
            Do not use LIMIT & IN/ALL/ANY/SOME subquery
            """

eval_prompt = """### Task
# Generate a SQL query to answer the following question. Give answer according to MySql 5.7.21:
# `{question}

# ### Database Schema
# This query will run on a database whose schema is represented in this string:
# if all columns are required give the (*) notation.
# Do not use LIMIT & IN/ALL/ANY/SOME subquery

# CREATE TABLE IF NOT EXISTS orders (
# <define your="" table="" schema="">;

# ### SQL
# Given the database schema, here is the SQL query that answers `{question}`:
# ```sql
# """.format(question=question)

sql = evaluate(question)
sql, if_sql = refine_sql(sql)
response = sql_to_gemini(sql, question, if_sql)
print_answer(response)


             Table: Employee
            Column Name: id, name, salary, departmentId
            Type: int, varchar, int, int

            id is the primary key (column with unique values) for this table.
            departmentId is a foreign key (reference column) of the ID from the Department table.
            Each row of this table indicates the ID, name, and salary of an employee. It also contains the ID of their department.


            Table: Department
            Column Name: id, name
            Type: int, varchar

            id is the primary key (column with unique values) for this table.
            Each row of this table indicates the ID of a department and its name.


            A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.

            Write a javascript function to find the employees who